In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#1.Library




In [ ]:
# from tensorflow.keras.datasets import cifar10

In [ ]:
# def load_cifar10():
#     # load the data
#     (x_train, y_train), (x_test, y_test) = cifar10.load_data()
#     # normalize our inputs to be in the range[-1, 1]
#     x_train = (x_train.astype(np.float32) - 127.5)/127.5

#     x_train = x_train.reshape(50000, 3072)
#     return (x_train, y_train, x_test, y_test)

In [ ]:
# x_train, y_train, x_test, y_test = load_cifar10()

In [ ]:
# print (x_train.shape)

In [ ]:
import tensorflow as tf
import os
import glob
import helper
from functools import partial
import numpy as np
from tqdm import tqdm
import cv2
from skimage import io
from PIL import Image
import pickle as pkl
import scipy.misc
import random
import cv2
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
from tensorflow.keras.models import load_model
from tensorflow.keras import backend

#2.Data preprocessing

In [ ]:
DATA_BASE_DIR = '/content/drive/My Drive/data/gan_face/data/img_faces_stylegan/'
image_size = 128
random_dim = 100

In [ ]:
list_ds = tf.data.Dataset.list_files(DATA_BASE_DIR + '/*')

In [ ]:
for f in list_ds.take(5):
    print(f.numpy())

In [ ]:
def normalize(image):
    '''
        normalizing the images to [-1, 1]
    '''
    image = tf.cast(image, tf.float32)
    image = (image - 127.5) / 127.5
    return image

In [ ]:
def augmentation(image):
    ''' input: image ndarray
        output: augmanetated data
        Perform some augmentation
    '''
    image = tf.image.random_flip_left_right(image)
    return image

In [ ]:
def preprocess_image(file_path, target_size=512):
    ''' input: file path, size of output image
        output: images
        Load and resize image from file path
    '''
    images = tf.io.read_file(file_path)
    images = tf.image.decode_jpeg(images, channels=3)
    images = tf.image.resize(images, (target_size, target_size),
                           method='nearest', antialias=True)
    images = augmentation(images)
    images = normalize(images)
    return images

In [ ]:
files = glob.glob(os.path.join(DATA_BASE_DIR, '*.jpg'))

In [ ]:
print (type(files))

In [ ]:
image_batch_ath = random.sample(files ,10)

In [ ]:
X = []
for i in tqdm(range(len(files)),desc="load"):
  image = cv2.imread(files[i])
  image = cv2.resize(image,(64,64))
  image.flatten()
  X.append(image)        
X = [np.random.random((3072*4))
        for i in range(len(X))]

X = np.concatenate([arr[np.newaxis] for arr in X])
X.shape

In [ ]:
print (type(X))

In [ ]:
X.shape

In [ ]:
files = glob.glob(os.path.join(DATA_BASE_DIR, '*.jpg'))
X = []
for temp in files:
    img = cv2.imread(temp)
    img = cv2.resize(img,(512,512))
    X.append(img)
np.array(X)

X_train = X[:2000]
X_train = [np.random.random((512, 512, 3))
        for i in range(2000)]
X_train = np.concatenate([arr[np.newaxis] for arr in X_train])

#2.Generator And Discriminator

In [ ]:
def get_generator():
    generator = Sequential()
    generator.add(Dense(128, input_dim=random_dim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    generator.add(LeakyReLU(0.2))

    generator.add(Dense(256))
    generator.add(LeakyReLU(0.2))

    generator.add(Dense(512))
    generator.add(LeakyReLU(0.2))
    generator.add(Dense(1024))
    generator.add(LeakyReLU(0.2))
    generator.add(Dense(1024*2))
    generator.add(LeakyReLU(0.2))
    generator.add(Dense(1024*4))
    generator.add(LeakyReLU(0.2))
    
    
    generator.add(Dense(1024*3*4, activation='tanh'))
    generator.compile(loss='binary_crossentropy', optimizer= Adam( learning_rate=0.0001 , beta_1=0.5 ))
    return generator

In [ ]:
def get_discriminator():
    discriminator = Sequential()
    

    discriminator.add(Dense(1024*4, input_dim=3072*4, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(512*4))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(512*2))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(256))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(128))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense( 1 , activation='sigmoid'))
    discriminator.compile( loss='binary_crossentropy' , optimizer=Adam( learning_rate=0.0001 , beta_1=0.5 ))
    return discriminator

In [ ]:
def get_gan_network(discriminator, random_dim, generator):
    # We initially set trainable to False since we only want to train either the
    # generator or discriminator at a time
    discriminator.trainable = False
    # gan input (noise) will be 100-dimensional vectors
    gan_input = Input(shape=(random_dim,))
    # the output of the generator (an image)
    x = generator(gan_input)
    # get the output of the discriminator (probability if the image is real or not)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=Adam( learning_rate=0.0001 , beta_1=0.5 ))
    return gan

In [ ]:
def plot_generated_images(epoch, generator, examples=100, dim=(50, 50), figsize=(100, 100)):
    noise = np.random.normal(0, 1, size=[examples, random_dim])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, 64, 64, 3)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('/content/drive/My Drive/data/gan_face/data/output_gan/gan_generated_image_epoch_%d.png' % epoch)

In [ ]:
def plot_graph(d_losses,g_losses,epoch):
  plt.figure(figsize=(10,8))
  plt.plot(d_losses,label='D loss')
  plt.plot(g_losses, label='G loss')
  plt.title('Loss Graph Generator loss vs Discriminator loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()
  plt.savefig('/content/drive/My Drive/data/gan_face/data/graph/gan_loss_%d.png' % epoch)

In [ ]:
def saveModels(generator, discriminator, epoch):
    generator.save('/content/drive/My Drive/data/gan_face/data/model_gan_t1/gan_generator_epoch_%d.h5' % epoch)
    discriminator.save('/content/drive/My Drive/data/gan_face/data/model_gan_t1/gan_discriminator_epoch_%d.h5' % epoch)

In [ ]:
d_losses = []
g_losses = []

In [ ]:
def train (dataset,epochs=100 , batch_size=128):
    # Split the training data into batches of size 128
    #files = glob.glob(os.path.join(data_path, '*.jpg'))
    x_train = dataset[:2000]
    batch_count = len(files) / batch_size

    generator = get_generator()
    discriminator = get_discriminator()
    gan = get_gan_network(discriminator, random_dim, generator)

    for e in np.arange(1, epochs+1):
        print ('-'*15, 'Epoch %d' % e, '-'*15)
        for _ in tqdm(np.arange(batch_count)):
            # Get a random set of input noise and images
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])
            '''image_batch_path = random.sample(files,batch_size)
            image_batch = []
            for file in image_batch_path:
              image = cv2.imread(file)
              image = cv2.resize(image,(32,32))
              image_batch.append(image)

            image_batch = [np.random.random((3072))
            for i in range(len(image_batch)) ]
            image_batch = np.concatenate([arr[np.newaxis] for arr in image_batch])
            image_batch.reshape(batch_size,3072)'''

              
            image_batch = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]
            # Generate fake CIFAR images
            generated_images = generator.predict(noise)
            print (image_batch.shape)
            X = np.concatenate([image_batch, generated_images])

            # Labels for generated and real data
            y_dis = np.zeros(2*batch_size)
            # One-sided label smoothing
            y_dis[:batch_size] = 0.9

            # Train discriminator
            discriminator.trainable = True
            dloss = discriminator.train_on_batch(X, y_dis)

            # Train generator
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])
            y_gen = np.ones(batch_size)
            discriminator.trainable = False
            gloss = gan.train_on_batch(noise, y_gen)
            print([dloss,gloss])
            d_losses.append(dloss)
            g_losses.append(gloss)

        if e == 1 or e % 50 == 0:
            plot_generated_images(e, generator)
            plot_graph(d_losses, g_losses,e)
            saveModels(generator,discriminator,e)

#3.Train


In [ ]:
if __name__ == '__main__':
    train(X,200, 256)